# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
!pip install gmaps

In [43]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import gmaps.datasets

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [44]:
weather_data = pd.read_csv('../WeatherPy/output_data/cities.csv')
weather_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,manacapuru,-3.2997,-60.6206,77.00,94,0,1.14,BR,1610419474
1,1,hermanus,-34.4187,19.2345,68.00,91,85,3.00,ZA,1610419410
2,2,cape town,-33.9258,18.4232,70.00,73,20,8.05,ZA,1610419475
3,3,vardo,70.3705,31.1107,19.92,95,40,24.63,NO,1610419475
4,4,port elizabeth,-33.9180,25.5701,68.00,94,90,11.50,ZA,1610419475
...,...,...,...,...,...,...,...,...,...,...
559,559,jucas,-6.5253,-39.5275,75.90,76,100,4.43,BR,1610419540
560,560,los andes,-32.8337,-70.5983,69.80,43,0,5.75,CL,1610419346
561,561,gedo,9.0167,37.4500,48.42,87,29,2.46,ET,1610419540
562,562,filiates,39.6011,20.3119,57.00,100,100,3.44,GR,1610419540


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [45]:
#Access maps with unique API key
gmaps.configure(api_key = g_key)

In [46]:
#Store latitude and longitude in locations
locations = weather_data[['Lat', 'Lng']]

humidity = weather_data["Humidity"].astype(float)

In [47]:
#Plot Heatmap
fig = gmaps.figure(center = (35.0, 0.0), zoom_level=1.5)
max_humidity = np.max(humidity)

#Create heat layer
#heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 2, opacity = 0.5)
#fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [48]:
#Narrow the dataframe
narrowed_city_df = weather_data.loc[(weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80) & (weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,manacapuru,-3.2997,-60.6206,77.00,94,0,1.14,BR,1610419474
13,13,diapaga,12.0708,1.7889,72.59,20,0,8.01,BF,1610419476
27,27,mataura,-46.1927,168.8643,77.88,33,0,1.14,NZ,1610419477
142,142,takoradi,4.8845,-1.7554,78.31,88,0,9.62,GH,1610419488
178,178,axim,4.8699,-2.2405,79.07,86,0,9.69,GH,1610419491
207,207,sembe,1.6481,14.5806,72.55,80,0,1.36,CG,1610419494
228,228,pochutla,15.7432,-96.4661,75.00,76,0,1.01,MX,1610419496
349,349,filingue,14.3521,3.3168,70.90,20,0,9.31,NE,1610419511
369,369,caravelas,-17.7125,-39.2481,75.11,88,0,6.53,BR,1610419513
393,393,conceicao da barra,-18.5933,-39.7322,74.79,88,0,5.55,BR,1610419518


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
#Create a hotel dataframe
hotel_df = pd.DataFrame(narrowed_city_df).reset_index()
hotel_df = hotel_df.drop(hotel_df.columns[0], axis=1)

#Add a "Hotel Name" column
hotel_df["Hotel Name"] = ""
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,manacapuru,-3.2997,-60.6206,77.00,94,0,1.14,BR,1610419474,
1,13,diapaga,12.0708,1.7889,72.59,20,0,8.01,BF,1610419476,
2,27,mataura,-46.1927,168.8643,77.88,33,0,1.14,NZ,1610419477,
3,142,takoradi,4.8845,-1.7554,78.31,88,0,9.62,GH,1610419488,
4,178,axim,4.8699,-2.2405,79.07,86,0,9.69,GH,1610419491,
5,207,sembe,1.6481,14.5806,72.55,80,0,1.36,CG,1610419494,
6,228,pochutla,15.7432,-96.4661,75.00,76,0,1.01,MX,1610419496,
7,349,filingue,14.3521,3.3168,70.90,20,0,9.31,NE,1610419511,
8,369,caravelas,-17.7125,-39.2481,75.11,88,0,6.53,BR,1610419513,
9,393,conceicao da barra,-18.5933,-39.7322,74.79,88,0,5.55,BR,1610419518,


In [50]:
#Search for hotels with 5000 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set parameters to search
params = {
         "type" : "hotel",
         "keyword" : "hotel",
         "radius" : 5000,
         "key" : g_key
}

In [52]:
count = 0

for index, row in hotel_df.iterrows():
    
    #Get city name from the list
    city_name = row["City"]
    lat = row['Lat']
    lng = row['Lng']
    
    #add params
    params['location'] = f'{lat},{lng}'
    
    count += 1
    
    print(f"Retrieving Results for: {city_name}.")
    
    #Make request
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
        print(f"Hotel Found")
        print("------------")
        
    except:
        hotel_df.loc[index, "Hotel Name"] = "No Hotel in Range"
        print("Missing field/result... skipping.")
        print("------------")
        
        #set sleep time to avoid error
        time.sleep(1.00)
        
#Once search complete
print("Searching Complete")

Retrieving Results for: manacapuru.
Hotel Found
------------
Retrieving Results for: diapaga.
Hotel Found
------------
Retrieving Results for: mataura.
Hotel Found
------------
Retrieving Results for: takoradi.
Hotel Found
------------
Retrieving Results for: axim.
Hotel Found
------------
Retrieving Results for: sembe.
Missing field/result... skipping.
------------
Retrieving Results for: pochutla.
Hotel Found
------------
Retrieving Results for: filingue.
Missing field/result... skipping.
------------
Retrieving Results for: caravelas.
Hotel Found
------------
Retrieving Results for: conceicao da barra.
Hotel Found
------------
Retrieving Results for: beloha.
Missing field/result... skipping.
------------
Retrieving Results for: lazaro cardenas.
Hotel Found
------------
Searching Complete


In [53]:
#Store hotel results
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,manacapuru,-3.2997,-60.6206,77.00,94,0,1.14,BR,1610419474,Hotel Amazon Experience
1,13,diapaga,12.0708,1.7889,72.59,20,0,8.01,BF,1610419476,Diapaga
2,27,mataura,-46.1927,168.8643,77.88,33,0,1.14,NZ,1610419477,Falls Hotel
3,142,takoradi,4.8845,-1.7554,78.31,88,0,9.62,GH,1610419488,Best Western Plus Atlantic Hotel
4,178,axim,4.8699,-2.2405,79.07,86,0,9.69,GH,1610419491,Ankobra Beach Resort
5,207,sembe,1.6481,14.5806,72.55,80,0,1.36,CG,1610419494,No Hotel in Range
6,228,pochutla,15.7432,-96.4661,75.00,76,0,1.01,MX,1610419496,Hotel Posada San Jose
7,349,filingue,14.3521,3.3168,70.90,20,0,9.31,NE,1610419511,No Hotel in Range
8,369,caravelas,-17.7125,-39.2481,75.11,88,0,6.53,BR,1610419513,Mar Doce de Abrolhos
9,393,conceicao da barra,-18.5933,-39.7322,74.79,88,0,5.55,BR,1610419518,Porto Marlin Hotel


In [54]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [55]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))